# **Assignment 3: Estimating a Search Model**

## **Applied Econometrics**

### Conor Bayliss

In this homework, we are going to estimate the parameters of the search model for each demographic group *individually*. That is, you will *not* impose the parametrics restrictions that mapped demographics $X$ to deeper parameters using the `NamedTuples` I used last week.

First, import the necessary packages.

In [1]:
using LinearAlgebra, QuadGK, Distributions, CSV, DataFrames, DataFramesMeta, Statistics, Optim, FastGaussQuadrature, ForwardDiff, Roots

#### **Adapting Code for Automatic Differentiation**

`QuadGK` doesn't play nicely with automatic differentiation since it adjusts the number of nodes adaptively. One solution is to use a fixed number of nodes and weights with `FastGaussQuadrature`. Here is a simple example.

In [2]:
function integrandGL(f,a,b;num_nodes = 10)
    nodes, weights = gausslegendre(num_nodes)
    ∫f = 0.
    for k in eachindex(nodes)
        x = (a+b)/2 + (b-a)/2*nodes[k]
        ∫f += weights[k]*f(x)
    end
    return ∫f*(b-a)/2
end

dS(x;F,β,δ) = (1-cdf(F,x)) / (1-β*(1-δ))
res_wage(wres, b,λ,δ,β,F) = wres - b - β * λ * integrandGL(x -> dS(x;F,β,δ),wres,quantile(F,0.999))
ForwardDiff.derivative(wres -> res_wage(wres,0.,0.5,0.03,0.99,LogNormal(0.,1.)),1.)

7.235638422590369

#### **Re-writing the model solution**

Based on this, we're going to re-write the model solution using this new integration routine. We will also use `Roots` to solve for the reservation wage in a way that will also play nicely with `ForwardDiff`.

In [3]:
res_wage_solution(wres, b,λ,δ,β,F::Distribution) = wres - b - β * λ * integrandGL(x -> dS(x;F,β,δ),wres,quantile(F,0.999))
pars = (;b=-5., λ=0.45,δ=0.03,β=0.99, F=LogNormal(1.,1.))

function solve_res_wage(b,λ,δ,β,F)
    return find_zero(wres -> res_wage_solution(wres,b,λ,δ,β,F),eltype(b)(4.))
end

solve_res_wage(0.,0.4,0.03,0.995,LogNormal())
ForwardDiff.derivative(x -> solve_res_wage(x,0.4,0.03,0.995,LogNormal()),0.)

0.4400135335598148

#### **Cleaning the Data**

The data cleaning is mostly the same as in **Assignment 2**. We add a function which pulls a `NamedTuple` out for a specific demographic group.

In [4]:
data = CSV.read("C:\\Users\\bayle\\Documents\\Github\\metrics\\hw2\\data\\cps_00019.csv",DataFrame)
data = @chain data begin
    @transform :E = :EMPSTAT.<21
    @transform @byrow :wage = begin
        if :PAIDHOUR==0
            return missing
        elseif :PAIDHOUR==2
            if :HOURWAGE<99.99 && :HOURWAGE>0
                return :HOURWAGE
            else
                return missing
            end
        elseif :PAIDHOUR==1
            if :EARNWEEK>0 && :UHRSWORKT<997 && :UHRSWORKT>0
                return :EARNWEEK / :UHRSWORKT
            else
                return missing
            end
        end
    end
    @subset :MONTH.==1
    @select :AGE :SEX :RACE :EDUC :wage :E :DURUNEMP
    @transform begin
        :bachelors = :EDUC.>=111
        :nonwhite = :RACE.!=100 
        :female = :SEX.==2
        :DURUNEMP = round.(:DURUNEMP .* 12/52)
    end
end

# the whole dataset in a named tuple
wage_missing = ismissing.(data.wage)
wage = coalesce.(data.wage,1.)
N = length(data.AGE)
X = [ones(N) data.bachelors data.female data.nonwhite]
# create a named tuple with all variables to conveniently pass to the log-likelihood:
d = (;logwage = log.(wage),wage_missing,E = data.E,tU = data.DURUNEMP, X) #<- you will need to add your demographics as well.

function get_data(data,C,F,R)
    data = @subset data :bachelors.==C :female.==F :nonwhite.==R
    wage_missing = ismissing.(data.wage)
    wage = coalesce.(data.wage,1.)
    N = length(data.AGE)
    # create a named tuple with all variables to conveniently pass to the log-likelihood:
    return d = (;logwage = log.(wage),wage_missing,E = data.E,tU = data.DURUNEMP) 
end

dx = get_data(data,1,0,0) #<- data for white men with a college degree
@show typeof(dx)

typeof(dx) = @NamedTuple{logwage::Vector{Float64}, wage_missing::BitVector, E::BitVector, tU::Vector{Float64}}


@NamedTuple{logwage::Vector{Float64}, wage_missing::BitVector, E::BitVector, tU::Vector{Float64}}

Note that `dx` is our instance of the data for white men with a college degree. It is saved as a `NamedTuple`.

### **Part 1**

*Fix $\sigma_{\zeta}$ (the standard deviation of measurement error in log wages) to 0.05. Following your work from last week (and recitation this week) write a function that calculates the log-likelihood of a single month of data from the CPS given $(h,\delta,\mu,\sigma,w^*)$ where $w^*$ is the reservation wage and $h =$ $\lambda$ x $(1-F_W(w^*;\mu,\sigma))$.*

First, let us define $\phi$ and $\Phi$, the pdf and cdf respectively of a Normal distribution with mean $\mu$ and standard deviation $\sigma$.

In [5]:
ϕ(x,μ,σ) = pdf(Normal(μ,σ),x)
Φ(x,μ,σ) = cdf(Normal(μ,σ),x)

Φ (generic function with 1 method)

Now, write a function for the log-likelihood of observed wages. Remember that we need to integrate out measurement error. Recall that the likelihood of an observed wage $W^o$ is:
$$
f(W^o|E,X) = \int_{w^*} \frac{\phi(\log(w);\mu,\sigma)}{1-\Phi(\log(w^*);\mu,\sigma)}\phi(\log(W^o)-w;\sigma_\zeta)dw

$$

In [6]:
function logwage_likelihood(logwage, F::Distribution, σζ,wres)
    f(x) = pdf(F,x) / (1-cdf(F,wres)) * ϕ(logwage,log(x),σζ) 
    ub = quantile(F,0.9999)
    return integrandGL(f,wres,ub)
end

logwage_likelihood (generic function with 1 method)

Now that we have integrated out measurement error, let us get the log-likelihhod of a single observation.

In [7]:
function log_likelihood(d::NamedTuple, pars::NamedTuple,n)
    (;h,σζ,wres,F,δ) = pars
    ll = 0.
    if d.E[n]
        ll += log(h) - log(h+δ)
        if !d.wage_missing[n]
            ll += logwage_likelihood(d.logwage[n],F,σζ,wres)
        end
    else
        ll += log(δ) - log(h+δ)
        ll += log(h) + d.tU[n] * log(1-h)
    end
    return ll
end

log_likelihood (generic function with 1 method)

Finally, let us write a function which maps the vector x into parameters.

In [8]:
logit(x) = exp(x) / (1 + exp(x))
logit_inv(x) = log(x/(1-x))
function update(pars,x)
    h = logit(x[1])
    δ = logit(x[2])
    μ = x[3]
    σ = exp(x[4])
    wres = exp(x[5])
    F = LogNormal(μ,σ)
    σζ = 0.05
    β = 0.995
    return (; pars..., h,δ,μ,σ,wres,F,σζ,β)
end

update (generic function with 1 method)

Now, iterate over the dataset and calculate the log-likelihood.

In [9]:
function log_likelihood_obj(d::NamedTuple, pars::NamedTuple,x)
    pars = update(pars,x)
    ll = 0.
    for n in 1:length(d.logwage)
        ll += log_likelihood(d,pars,n)
    end
    return ll / length(d.E)
end

log_likelihood_obj (generic function with 1 method)

### **Part 2**

*Use the log-likelihood to get maximum likelihood estimates of $(\hat{h},\hat{\delta},\hat{\mu},\hat{\sigma},\hat{w^*})$ for *white men with a college degree*. What is the advantage of estimating $h$ and $w^*$ directly instead of $\lambda$ and $b$?*

We can now pass the above to `Optim`. Recall that `dx` is our instance of the data for white men with a college degree.

In [47]:
x0 = [logit_inv(0.5),logit_inv(0.03),2.,log(1.),log(5.)]
pars = (;σζ=0.05,β=0.995)
log_likelihood_obj(dx,pars,x0)
res = optimize(x -> -log_likelihood_obj(dx,pars,x),x0,BFGS(),Optim.Options(show_trace=false))
pars = update(pars,res.minimizer)
@show pars

pars = (σζ = 0.05, β = 0.995, h = 0.17641764362085174, δ = 0.0038283722601471946, μ = 2.2091536420274718, σ = 1.103509994766977, wres = 21.80490807041903, F = LogNormal{Float64}(μ=2.2091536420274718, σ=1.103509994766977))


(σζ = 0.05, β = 0.995, h = 0.17641764362085174, δ = 0.0038283722601471946, μ = 2.2091536420274718, σ = 1.103509994766977, wres = 21.80490807041903, F = LogNormal{Float64}(μ=2.2091536420274718, σ=1.103509994766977))

Therefore, our estimated parameters are, to 5 decimal places:

| Parameter | Estimate | 
| -------- | -------- | 
| $\hat{h}$   | 0.17642    |
| $\hat{\delta}$    | 0.00383    | 
| $\hat{\mu}$    | 2.20915    | 
| $\hat{\sigma}$    | 1.10351    | 
| $w^*$    | 21.80491    | 

### **Part 3**

*Back out the implied maximum likelihood estimates of $\hat{\lambda}$ and $\hat{b}$ as a function of the estimated parameters from **Part 1**.*

Recall from **Part 1** that $\lambda$ is given by:
$$
\lambda = \frac{h}{(1-F_W(w^*;\mu,\sigma))}
$$
and that we can obtain $b$ from the equation for the reservation wage:
$$
w^* = b + \beta \lambda \int_{w^*} \frac{1-F_W(w)}{1-\beta(1-\delta)}dw
$$
$$
\implies b = w^* - \beta \lambda \int_{w^*} \frac{1-F_W(w)}{1-\beta(1-\delta)}dw.
$$
The following code backs out the parameter values.

In [26]:
λ = pars.h / (1-cdf(pars.F,pars.wres))
@show λ
b = pars.wres - pars.β * λ * integrandGL(x -> dS(x;pars.F,pars.β,pars.δ),pars.wres,quantile(pars.F,0.999))
@show b

λ = 0.8226737936000635
b = -501.51164449347914


-501.51164449347914

So, our implicitly estimated parameters are:

| Parameter | Estimate | 
| -------- | -------- | 
| $\hat{\lambda}$   |  0.82267    |
| $\hat{b}$    | -501.51164    | 

### **Part 4**

*Provide an estimate of the asymptotic variance of $(\hat{h},\hat{\delta},\hat{\mu},\hat{\sigma},\hat{w^*})$ using the standard MLE formula.*

To find the standard errors, we can take advantage of the delta method. Specifically, it tells us that

In [22]:
H = ForwardDiff.hessian(x -> log_likelihood_obj(dx,pars,x),res.minimizer)
N = length(dx.E)
avar = inv(-H)
seh = sqrt(avar[1,1] / N)
@show seh
seδ = sqrt.(avar[2,2] / N)
@show seδ
seμ = sqrt(avar[3,3] / N)
@show seμ
seσ = sqrt(avar[4,4] / N)
@show seσ
sewres = sqrt(avar[5,5] / N)
@show sewres

seh = 0.07870792169024328
seδ = 0.09740231570648486
seμ = 0.17253358400531876
seσ = 0.042114134111141954
sewres = 0.00843753864798779


0.00843753864798779

Hence, we can present a table of estimates and their standard errors:
| Parameter | Estimate | Standard error |
| -------- | -------- | -------- |
| $\hat{h}$    | 0.17642      | 0.07871     |
| $\hat{\delta}$    | 0.00383    | 0.09740     |
| $\hat{\mu}$   | 2.20915   | 0.17253    |
| $\hat{\sigma}$    | 1.10351    | 0.04211 |
| $w^*$    | 21.80491    | 0.00844 |


### **Part 5**

*Recall that the delta method implies that if $\hat{\delta}$ is asymptotically normal with asymptotic variance $V$ then the vector-values function $F(\hat{\delta})$ is also asymptotically normal with:*
$$
\sqrt{N}F((\hat{\theta})-F(\theta)) \xrightarrow{d} \mathcal{N}(0,\nabla_{\theta'}FV\nabla_{\theta}F')
$$
*Use this fact to estimate the asymptotic variance of $(\hat{h},\hat{\delta},\hat{\mu},\hat{\sigma},\hat{w^*},\hat{\lambda},\hat{b})$.*

To do so, let us define a function `final_ests` which will take the estimated parameters and back out the other necessary parameters. We will then use it to find the Jacobian and estimate the variances using the Delta method.

In [35]:
function final_ests(x_est, pars)
    pars = update(pars,x_est)
    (;h,δ,μ,σ,wres,F,β) = pars
    λ = h / (1-cdf(F,wres))
    b = pars.wres - β * λ * integrandGL(x -> dS(x;F,β,δ),wres,quantile(F,0.999))
    return [h,δ,μ,σ,wres,λ,b]
end
@show final_ests(res.minimizer,pars)
dF = ForwardDiff.jacobian(x -> final_ests(x,pars),res.minimizer)
var_est = dF * avar * dF' / N
se = sqrt.(diag(var_est))
@show se

final_ests(res.minimizer, pars) = [0.17641764362085174, 0.0038283722601471946, 2.2091536420274718, 1.103509994766977, 21.80490807041903, 0.8226737936000635, -501.51164449347914]
se = [0.011435824872672693, 0.0003714647528974018, 0.17253358400531876, 0.04647336791260203, 0.18397975455998142, 0.14808326720908588, 25.495587911939875]


7-element Vector{Float64}:
  0.011435824872672693
  0.0003714647528974018
  0.17253358400531876
  0.04647336791260203
  0.18397975455998142
  0.14808326720908588
 25.495587911939875

Hence, we can now present our final results for white men with a college degree.

| Parameter | Estimate | Standard error |
| -------- | -------- | -------- |
| $\hat{h}$    | 0.17642      | 0.01144     |
| $\hat{\delta}$    | 0.00383    | 0.00037     |
| $\hat{\mu}$   | 2.20915   | 0.17253    |
| $\hat{\sigma}$    | 1.10351    | 0.04647 |
| $w^*$    | 21.80491    | 0.18398 |
| $\hat{\lambda}$    | 0.82267    | 0.14808 |
| $\hat{b}$    | -501.51164    | 25.49559 |

Note that the inconsistencies in the standard errors between the two methods are since one uses the Hessian and the other uses the Jacobian.

### **Part 6**

*Now report all of your estimates and standard errors for this group. Repeat this exercise for each group.*

*If we thought that the parametric relationship using $\gamma$ from **Assignment 2** described the true values of the parameters for each group, how might we use these group-specific estimates to derive estimates of each $\gamma$?*

Now, let's write a function to iterate over the relevant data subsets are obtain parameter estimates and standard errors for each.

In [67]:
function iterate(storage,pars)
    se = zeros(7,8)
    iter = 0
    for x in 0:1
        for y in 0:1
            for z in 0:1
                iter += 1
                d = get_data(data,x,y,z) ### get_data(data,C,F,R)
                res = optimize(x -> -log_likelihood_obj(d,pars,x),x0,BFGS(),Optim.Options(show_trace=false)) # find the MLE
                pars = update(pars,res.minimizer) # update the parameters
                storage[:,iter] = final_ests(res.minimizer,pars) # store the results
                dF = ForwardDiff.jacobian(x -> final_ests(x,pars),res.minimizer) # compute the jacobian
                var_est = dF * avar * dF' / N # compute the variance of the estimates using the Delta method
                se[:,iter] = sqrt.(diag(var_est)) # store the standard errors
            end
        end
    end
    return storage,se
end
                
params, errors = iterate(zeros(7,8),pars)

([0.211386140082968 0.15634047422619457 … 0.16957862408630428 0.15844155895412296; 0.010965423469391606 0.012984290419368327 … 0.003016762598031792 0.004600159483590965; … ; 0.4900830220673784 0.15634047422619457 … 0.34222293919326596 0.15851295800385384; -142.31236124926681 -308.1910997484958 … -496.74313438427237 -576.7534617651261], [0.013120771099203142 0.010381427714277715 … 0.01108378446450345 0.010494743588267076; 0.0010563459343237822 0.001248278723147437 … 0.00029295321847989665 0.00044600501040462694; … ; 0.07726794508643905 0.010381427714277715 … 0.046665900601739356 0.010500022518590856; 9.550968949453676 41.7945041477519 … 29.599186944629636 105.63062626379774])

Finally, store the estimates and the standard errors in a `DataFrame`.

In [68]:
df_out = DataFrame()

param_names = ["h","δ","μ","σ","wres","λ","b"]

df_out[!, :param_names] = param_names

for i in 1:8
    df_out[!, "params$i"] = params[:,i]
    df_out[!, "se$i"] = errors[:,i]
end
C,F,R = [0,0,0] 
demographic_names = ["W_M_NC", "NW_M_NC", "W_W_NC", "NW_W_NC", "W_M_C", "NW_M_C", "W_W_C", "NW_W_C"]

for (i, name) in enumerate(demographic_names)
    rename!(df_out, "params$i" => "p_$name", "se$i" => "se_$name")
end


df_out

Row,param_names,p_W_M_NC,se_W_M_NC,p_NW_M_NC,se_NW_M_NC,p_W_W_NC,se_W_W_NC,p_NW_W_NC,se_NW_W_NC,p_W_M_C,se_W_M_C,p_NW_M_C,se_NW_M_C,p_W_W_C,se_W_W_C,p_NW_W_C,se_NW_W_C
,String,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,h,0.211386,0.0131208,0.15634,0.0103814,0.202006,0.0126877,0.149288,0.00999598,0.176418,0.0114358,0.164141,0.0107986,0.169579,0.0110838,0.158442,0.0104947
2,δ,0.0109654,0.00105635,0.0129843,0.00124828,0.00845501,0.000816574,0.0112654,0.00108491,0.00382837,0.000371465,0.0057423,0.000556101,0.00301676,0.000292953,0.00460016,0.000446005
3,μ,1.73575,0.172534,2.97264,0.172534,2.59919,0.172534,2.60459,0.172534,2.20915,0.172534,2.65632,0.172534,2.30222,0.172534,3.48832,0.172534
4,σ,1.03869,0.0437437,1.09247,0.0460086,1.09193,0.0459856,1.09158,0.0459708,1.10351,0.0464734,0.874308,0.0368207,1.15834,0.0487827,0.615712,0.0259302
5,wres,6.78993,0.0572903,3.80433e-10,3.20992e-12,4.53163e-11,3.82358e-13,6.75238e-11,5.69735e-13,21.8049,0.18398,15.3961,0.129905,10.1272,0.0854488,4.23881,0.0357652
6,λ,0.490083,0.0772679,0.15634,0.0103814,0.202006,0.0126877,0.149288,0.00999598,0.822674,0.148083,0.353336,0.0630307,0.342223,0.0466659,0.158513,0.0105
7,b,-142.312,9.55097,-308.191,41.7945,-365.998,48.9209,-224.947,30.4314,-501.512,25.4956,-284.677,23.6099,-496.743,29.5992,-576.753,105.631
